# MODIS Land products <img align="right" src="../../resources/easi_logo.jpg">

_Work in progress_

#### Index
- [Overview](#Overview)
- [Setup (dask, imports, query)](#Setup)
- [Product definition (measurements, flags)](#Product-definition)
- [Quality layer (mask)](#Quality-layer)
- [Scaling and nodata](#Scaling-and-nodata)
- [Visualisation](#Visualisation)
- [Appendix](#Appendix)

## Overview

MDOSI Land products are produced and managed by Land Processes DAAC (LPDAAC) hosted by USGS, https://lpdaac.usgs.gov.

#### Data source and documentation

Selected MODIS land products are ordered and downloaded from the AppEEARS API, https://lpdaacsvc.cr.usgs.gov/appeears. The AppEEARS MODIS products are merged from the hXXvYY HDF tiles, for a given bounding box, and converted to Geotiff. The original sinusoidal projection is retained.

Not all MODIS land products are available via AppEEARS, e.g. MCD43A2, so we expect to supplement the AppEEARS workflow with direct access to the LPDAAC data pool. We do not anticipate any differences in the EASI products from the two sources (but we'll cross that bridge when we come to it).

| Name | Product | Information |
|--|--|--|
| MOD11A1.006 Land surface temperature & emmissivity | lpdaac_mod11a1c6_lste | https://doi.org/10.5067/MODIS/MOD11A1.006 |
| MCD12Q1.006 Land cover type | lpdaac_mcd12q1c6_lctype | https://doi.org/10.5067/MODIS/MCD12Q1.006 |
| MOD13Q1.006 Vegetation indicies | lpdaac_mod13q1c6_vi | https://doi.org/10.5067/MODIS/MOD13Q1.006 |
| MCD43A4.006 NBAR | lpdaac_mcd43a4c6_nbar | https://doi.org/10.5067/MODIS/MCD43A4.006 |

#### EASI pipeline

| Task | Summary |
|------|---------|
| Source | AppEEARS|
| Download | Merged sinusoidal geotiff|
| Preprocess | |
| Format | Convert to COGs|
| Prepare | Y|
| TODO | |

### Product details
Used below in various places.

TODO:
- Move this to a function to simplify the notebook
- Add details for `lpdaac_mod11a1c6_lste` and `	lpdaac_mcd43a4c6_nbar`

In [ ]:
product_map = {}

# lpdaac_mod13q1c6_ndvi

product_map['lpdaac_mod13q1c6_vi'] = {
    'ndvi': {'scale': 0.0001, 'offset': 0, 'clim': (0,1)},
    'evi': {'scale': 0.0001, 'offset': 0, 'clim': (0,1)},
    'surf_refl_red': {'scale': 0.0001, 'offset': 0, 'clim': (0,1)},
    'surf_refl_nir': {'scale': 0.0001, 'offset': 0, 'clim': (0,1)},
    'surf_refl_blue': {'scale': 0.0001, 'offset': 0, 'clim': (0,1)},
    'surf_refl_mir': {'scale': 0.0001, 'offset': 0, 'clim': (0,1)},
    'view_zenith': {'scale': 0.01, 'offset': 0, 'clim': (0,180)},
    'sun_zenith': {'scale': 0.01, 'offset': 0, 'clim': (0,180)},
    'relative_azimuth': {'scale': 0.01, 'offset': 0, 'clim': (-180,180)},
    'composite_doy': {'scale': 1, 'offset': 0, 'clim': (1,366)},
    'quality': {'scale': 1, 'offset': 0, 'clim': (0,65534)},
    'pixel_reliability': {'scale': 1, 'offset': 0, 'clim': (0,3)},
    'flag': {
        'name': 'pixel_reliability',
        'good': ('good_data', 'marginal_data'),
        'bad': None,
    },
    'load': {
#         'crs': 'native',
#         'resolution': (231.656358263888990, 231.656358263888990),  # sinusoidal, 250 m
        'crs': 'epsg:4326',
        'resolution': (0.0025, 0.0025),  # wgs84, 250 m
    },
}

# lpdaac_mcd12q1c6_lctype

product_map['lpdaac_mcd12q1c6_lctype'] = {
    'lc_type1': {'scale': 1, 'offset': 0, 'clim': (1,17)},
    'lc_type2': {'scale': 1, 'offset': 0, 'clim': (0,15)},
    'lc_type3': {'scale': 1, 'offset': 0, 'clim': (0,10)},
    'lc_type4': {'scale': 1, 'offset': 0, 'clim': (0,8)},
    'lc_type5': {'scale': 1, 'offset': 0, 'clim': (0,11)},
    'lc_prop1': {'scale': 1, 'offset': 0, 'clim': (1,43)},
    'lc_prop2': {'scale': 1, 'offset': 0, 'clim': (1,40)},
    'lc_prop3': {'scale': 1, 'offset': 0, 'clim': (1,51)},
    'lc_prop1_assessment': {'scale': 1, 'offset': 0, 'clim': (0,100)},
    'lc_prop2_assessment': {'scale': 1, 'offset': 0, 'clim': (0,100)},
    'lc_prop3_assessment': {'scale': 1, 'offset': 0, 'clim': (0,100)},
    'quality': {'scale': 1, 'offset': 0, 'clim': (0,10)},
    'lw_mask': {'scale': 1, 'offset': 0, 'clim': (1,2)},
    'flag': {
        'name': 'quality',
        'good': ('classified_land',),
        'bad': None,
    },
    'load': {
#         'crs': 'native',
#         'resolution': (),  # sinusoidal, 500 m
        'crs': 'epsg:4326',
        'resolution': (0.005, 0.005),  # wgs84, 500 m
    },
}

## Setup

#### Dask

In [ ]:
from dask.distributed import Client

client = Client("tcp://10.0.35.181:40115")
client

#### Imports

In [ ]:
# Data tools
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime

# Datacube
import datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from odc.algo import xr_reproject   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_warp.py
from datacube.utils.geometry import GeoBox, box  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/geometry/_base.py
from datacube.utils.rio import configure_s3_access

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# import geoviews as gv
# from holoviews.operation.datashader import rasterize
hv.extension('bokeh', logo=False)

# Python
import sys, os, re

# Optional EASI tools
sys.path.append(os.path.expanduser('~/hub-notebooks/scripts'))
import notebook_utils

#### ODC database

In [ ]:
# For development products:
#  - This is a development ODC database while we test and demo this product.
CONF = """
[datacube]
db_hostname: v2-db-easihub-csiro-eks.cluster-ro-cvaedcg0qvwd.ap-southeast-2.rds.amazonaws.com
db_database: user_dev_odc
db_username: user
db_password: secretpassword
"""
from datacube.config import read_config, LocalConfig
dc = datacube.Datacube(config=LocalConfig(read_config(CONF)), env='datacube')

# dc = datacube.Datacube()

#### Example query

Change any of the parameters in the query object below to adjust the location, time, projection, or spatial resolution of the returned datasets.

Use the Explorer interface to check the temporal and spatial coverage for each product:
- https://explorer.csiro.easi-eo.solutions  + /product (when available)

In [ ]:
# Area name
min_longitude, max_longitude = (100, 120)
min_latitude, max_latitude = (-5, 10)
min_date = '2000-01-01'
max_date = '2022-12-31'

# Select product
# product = 'lpdaac_mod11a1c6_lste'
# product = 'lpdaac_mcd12q1c6_lctype'
product = 'lpdaac_mod13q1c6_vi'
# product = 'lpdaac_mcd43a4c6_nbar'

# Form an initial query object
query = {
    'product': product,                     # Product name
    'x': (min_longitude, max_longitude),    # "x" axis bounds
    'y': (min_latitude, max_latitude),      # "y" axis bounds
    'time': (min_date, max_date),           # Any parsable date strings
}

# CRS
native_crs = product_map[product]['load']['crs']
if native_crs == 'native':
    pass
    # TODO: Sinusoidal (first try) introduced an eastward shift of a few kms
    # Most common CRS
    # native_crs = notebook_utils.mostcommon_crs(dc, query)
    # display(native_crs)
    # query.update({
    #     'output_crs': native_crs,               # EPSG code
    #     'resolution': (-231.656358263888990, 231.656358263888990), # Target resolution
    #     'group_by': 'solar_day',                # Scene ordering
    #     'dask_chunks': {'x': 2048, 'y': 2048},  # Dask chunks
    # })

# Finalise query object
query.update({
    'output_crs': native_crs,               # EPSG code
    'resolution': product_map[product]['load']['resolution'], # Target resolution
    'group_by': 'solar_day',                # Scene ordering
    'dask_chunks': {'longitude': 2048, 'latitude': 2048},  # Dask chunks
})

In [ ]:
# Optional. Some products require AWS S3 credentials to supplied

# S3 credentials - required for s2_l2a
# configure_s3_access(aws_unsigned=True,requester_pays=False,client=client)
# print("Configured s3 requester pays data access")

In [ ]:
# Load data
data = dc.load(**query)

notebook_utils.heading(notebook_utils.xarray_object_size(data))
display(data)

# Calculate valid (not nodata) masks for each layer
valid_mask = masking.valid_data_mask(data)
notebook_utils.heading('Valid data masks for each variable')
display(valid_mask)

## Product definition

Display the measurement definitions for the selected product.

Use `list_measurements` to show the details for a product, and `masking.describe_variable_flags` to show the flag definitions.

In [ ]:
# Measurement definitions for the selected product
measurement_info = dc.list_measurements().loc[query['product']]
notebook_utils.heading(f'Measurement table for product: {query["product"]}')
notebook_utils.display_table(measurement_info)

# Separate lists of measurement names and flag names
measurement_names = measurement_info[ pd.isnull(measurement_info.flags_definition)].index
flag_names        = measurement_info[pd.notnull(measurement_info.flags_definition)].index

notebook_utils.heading('Selected Measurement and Flag names')
notebook_utils.display_table(pd.DataFrame({
    'group': ['Measurement names', 'Flag names'],
    'names': [', '.join(measurement_names), ', '.join(flag_names)]
}))

# Flag definitions
for flag in flag_names:
    notebook_utils.heading(f'Flag definition table for flag name: {flag}')
    notebook_utils.display_table(masking.describe_variable_flags(data[flag]))

## Quality layer

In [ ]:
# Make flags image
flag_name = product_map[product]['flag']['name']
flag_data = data[[flag_name]].persist() #.where(valid_mask[flag_name]).persist()   # Dataset
display(flag_data)

In [ ]:
# These options manipulate the color map and colorbar to show the categories for this product
options = {
    'title': f'Flag data for: {query["product"]} ({flag_name})',
    'cmap': cc.rainbow,
    'colorbar': True,
    'width': 800,
    'height': 450,
    'aspect': 'equal',
    'tools': ['hover'],
}

# Set the Dataset CRS
if native_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()
    x_dim, y_dim = ('longitude', 'latitude')
else:
    plot_crs = ccrs.Sinusoidal()
    x_dim, y_dim = ('x', 'y')


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used

quality_plot = flag_data.hvplot.image(
    x = x_dim, y = y_dim,         # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mode(),          # Datashader selects mode value, requires 'hv.Image'
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                          # Dataset CRS
    projection = ccrs.PlateCarree(),         # Output Projection (ccrs.PlateCarree() when coastline=True)
    coastline = '10m',                       # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist()

# display(quality_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(quality_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

In [ ]:
# Create mask layer

if product_map[product]['flag']['good']:
    good_pixel_flags = product_map[product]['flag']['good']
    pixel_mask = enum_to_bool(data[flag_name], good_pixel_flags)  # -> bool array
    display(f'Using good pixel flags: {good_pixel_flags}')
    
elif product_map[product]['flag']['bad']:
    bad_pixel_flags = product_map[product]['flag']['bad']
    pixel_mask = enum_to_bool(data[flag_name], bad_pixel_flags, invert=True)  # -> bool array
    display(f'Using bad pixel flags: {bad_pixel_flags}')
    
else:
    print('Define the "good" or "bad" flag set in Product details')

display(pixel_mask)  # Type: bool

## Masking and Scaling

In [ ]:
# Select a layer and apply masking and scaling, then persist in dask

layer_name = 'ndvi' # NDVI
# layer_name = 'lc_type1'  # LC Type

scale = 1
offset = 0
if 'scale' in product_map[product][layer_name]:
    scale = product_map[product][layer_name]['scale']
if 'offset' in product_map[product][layer_name]:
    offset = product_map[product][layer_name]['offset']

# Apply valid mask and good pixel mask
layer = data[[layer_name]].where(valid_mask[layer_name] & pixel_mask) * scale + offset
layer = layer.persist()

## Visualisation

In [ ]:
# Generate a plot

# Colour limit
clim = (0,1)
if 'clim' in product_map[product][layer_name]:
    clim = product_map[product][layer_name]['clim']

options = {
    'title': f'{query["product"]}: {layer_name}',
    'width': 800,
    'height': 450,
    'aspect': 'equal',
    'cmap': cc.rainbow,
    'clim': clim,                          # Limit the color range depending on the layer_name
    'colorbar': True,
    'tools': ['hover'],
}

# Set the Dataset CRS
if native_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()
    x_dim, y_dim = ('longitude', 'latitude')
else:
    plot_crs = ccrs.Sinusoidal()
    x_dim, y_dim = ('x', 'y')


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used

layer_plot = layer.hvplot.image(
    x = x_dim, y = y_dim,                        # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mode(),          # Datashader selects mean value
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                        # Dataset crs
    projection = ccrs.PlateCarree(),         # Output projection (use ccrs.PlateCarree() when coastline=True)
    coastline='10m',                         # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = options['clim'])

# display(layer_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(layer_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

## Appendix